In [ ]:
import pipulate as gs
from itertools import cycle
import requests
import re

use_proxies = False

proxy_list = list()
bad_blocks = list()

try:
    with open('goodproxies.txt') as proxies:
        for line in proxies:
            proxy_list.append(line[:-1])
    with open('badproxies.txt') as proxies:
        for line in proxies:
            bad_blocks.append(line[:-1])
    proxy_cycler = cycle(proxy_list)
    use_proxies = True
except:
    pass

print("Use proxies: %s" % use_proxies)

common_agents = list()
with open('useragents.txt') as agents:
    for line in agents:
        common_agents.append(line[:-1])
agent_cycler = cycle(common_agents)

In [ ]:
def fu():
    return 'bar'

In [ ]:
def ga_url(gaid, start, end, path):
    """Return Google Analytics metrics for a URL."""
    
    # OMG! How do I set all these parameters? Explore at:
    # https://ga-dev-tools.appspot.com/query-explorer/
    path = path.replace(',', r'\,')
    metrics = ['users', 
               'newUsers', 
               'sessions', 
               'bounceRate',
               'pageviewsPerSession', 
               'avgSessionDuration']
    metrics = ''.join(['ga:%s,' % x for x in metrics])[:-1]
    ga_request = gs.ga().data().ga().get(
        ids='ga:'+gaid,
        start_date=start,
        end_date=end,
        metrics=metrics,
        dimensions='ga:pagePath',
        sort='-ga:users',
        filters='ga:pagePath=='+path,
        segment='sessions::condition::ga:medium==organic',
        max_results=1
    )
    ga_response = ga_request.execute()
    return_value = []
    if 'rows' in ga_response:
        raw_rows = ga_response['rows'][0]
        return_value = raw_rows
    return return_value

In [ ]:
def gsc_keyword(prop, start, end, query):
    """Return position, clicks & impressions from GSC for keyword."""
    
    request = {
      "startDate": start,
      "endDate": end,
      "dimensions": [
        "query"
      ],
      "dimensionFilterGroups": [
        {
          "filters": [
            {
              "operator": "equals",
              "expression": query,
              "dimension": "query"
            }
          ]
        }
      ]
    }
    data = gs.gsc().searchanalytics().query(siteUrl='https://www.pcmag.com', body=request).execute()
    val = []
    if 'rows' in data:
        r = data['rows'][0]
        val = [start, end] + [r['keys'][0]] + [
            r['position']] + [r['clicks']] + [r['impressions']] + [r['ctr']]
    return val

In [ ]:
def gsc_url(prop, start, end, url):
    """Return position, clicks & impressions from GSC for URL."""

    request = {
      "startDate": start,
      "endDate": end,
      "dimensions": [
        "page"
      ],
      "dimensionFilterGroups": [
        {
          "filters": [
            {
              "operator": "equals",
              "expression": url,
              "dimension": "page"
            }
          ]
        }
      ]
    }
    data = gs.gsc().searchanalytics().query(siteUrl='https://www.pcmag.com', body=request).execute()
    val = []
    if 'rows' in data:
        r = data['rows'][0]
        val = [start, end] + [r['keys'][0]] + [
            r['position']] + [r['clicks']] + [r['impressions']] + [r['ctr']]
    return val

In [ ]:
def gsc_url_keyword(prop, start, end, query, url):
    """Return position, clicks & impressions from GSC for keyword for URL."""

    request = {
      "startDate": start,
      "endDate": end,
      "dimensions": [
        "query",
        "page"
      ],
      "dimensionFilterGroups": [
        {
          "filters": [
            {
              "operator": "equals",
              "expression": url,
              "dimension": "page"
            },
            {
              "operator": "equals",
              "expression": query,
              "dimension": "query"
            }
          ]
        }
      ]
    }
    data = gs.gsc().searchanalytics().query(siteUrl=prop, body=request).execute()
    val = []
    if 'rows' in data:
        r = data['rows'][0]
        val = [start] + [r['keys'][0]] + [r['keys'][1]] + [
            r['position']] + [r['clicks']] + [r['impressions']] + [r['ctr']]
    return val

In [ ]:
def serp(keyword, rpp=20):
    from urllib.parse import quote_plus
    global use_proxies, proxy_cycler, bad_blocks, agent_cycler
            
    print("Keyword: %s" % keyword)
    search_url = 'https://www.google.com/search?num=%s&q=%s' % (rpp, quote_plus(keyword))
    pattern = re.compile('<h3 class="r"><a href="(http.*?)"')
    landing_pages = []
    proxied_retries = 3
    naked_retries = 1
    
    # The preferred path if you have anonymous web proxies.
    # The light that burns twice as bright burns half as long.
    if use_proxies:
        for x in range(10):
            ip = next(proxy_cycler)
            if ip not in bad_blocks:
                break
            print("Couldn't find good proxy. Clear cache and try later.")
            raise SystemExit()
        block = '.'.join(ip.split('.')[:3])
        proxy = {'http': 'http://'+ip, 'https': 'https://'+ip}
        print('Using proxy: %s' % proxy)
        agent = next(agent_cycler)
        user_agent = {'User-agent': agent}
        for i in range(proxied_retries):
            try:
                response = requests.get(search_url, proxies=proxy, headers=user_agent)
                landing_pages = re.findall(pattern, response.text)
                break
            except:
                bad_blocks.append(ip)
                agent = next(agent_cycler)
                user_agent = {'User-agent': agent}
                for x in range(10):
                    ip = next(proxy_cycler)
                    if ip not in bad_blocks:
                        break
                print("Ran out of good proxies.")
                raise SystemExit()
    else:
        for i in range(naked_retries):
            try:
                response = requests.get(search_url, headers=user_agent)
                landing_pages = re.findall(pattern, response.text)
                break
            except:
                agent = next(agent_cycler)
                user_agent = {'User-agent': agent}
        
    return landing_pages